写一个cache装饰器， 允许过期， 但没有换出，没有清除 （45分钟完成）

* cache的必要元素： key -> value
* 超时时间

In [1]:
import inspect

In [2]:
def add(x, y):
    pass

In [4]:
sig = inspect.signature(add)

In [7]:
for k in sig.parameters.keys():
    print(k)

x
y


In [34]:
from functools import wraps
import inspect
import datetime

def cache(exp=0):
    def _cache(fn): ## fn cache
        cache = {}
        @wraps(fn)
        def wrap(*args, **kwargs):
            # TODO key 如何拼装
            key = []
            names = set()
            params = inspect.signature(fn).parameters
            for i, arg in enumerate(args):
                name = list(params.keys())[i]
                key.append((name, arg))
                names.add(name)
            key.extend(kwargs.items())
            names.update(kwargs.keys())
            for k, v in params.items():
                if k not in names:
                    key.append((k, v.default))
            key.sort(key=lambda x: x[0])
            key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])
            print(key)
            if key in cache.keys():
                ret, timestamp = cache[key]
                if exp == 0 or datetime.datetime.now().timestamp() - timestamp < exp:
                    print('cache hit')
                    return ret
            ret = fn(*args, **kwargs)
            print('cache miss')
            cache[key] = (ret, datetime.datetime.now().timestamp())
            return ret
        return wrap
    return _cache

In [35]:
@cache(5)
def add(x, y=3):
    return x + y

In [39]:
add(1, 3)

x=1&y=3
cache miss


4

写一个通用的命令分发器， 程序员可以方便的注册函数到某个命令， 当用户输入某个命令时， 路由到注册的函数， 如果此命令没有注册函数，执行默认函数

In [42]:
input('>>')

>>test


'test'

In [53]:
def command():
    commands = {}

    def register(command):
        def _register(fn):
            if command in commands:
                raise Exception('command {} exist'.format(command))
            commands[command] = fn
            return fn
        return _register

    def default_fn():
        print('unkonwn command')

    def run():
        while True:
            cmd = input('>>')
            if cmd.strip() == 'quit':
                return
            commands.get(cmd.strip(),default_fn)()
    
    return register, run
    

In [54]:
register, run = command()

In [55]:
@register('papa')
def papa():
    print('papa')

In [48]:
register('papa', papa)

In [56]:
run()

>>papa
papa
>>haha
unkonwn command
>>quit


## 装饰器的用途

AOP

针对一类问题做处理

与具体业务逻辑无关

常见的装饰器使用场景：

监控、缓存、路由、权限、参数检查、审计